## Initialize

In [1]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import requests

import fitz


from PyPDF2 import PdfFileReader

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()



## Choose file folder

In [2]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
options.remove('notebooks')
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Select:', options=('externDocuments', 'image', 'internDocuments'), value=None)

## Take all files from folder

In [15]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            filename = filename.replace('.pdf', '')

            # encode to base64
            encoded_string = base64.b64encode(f.read())
            encoded_doc = encoded_string.decode('ascii')

            # Get file info
            pdf = PdfFileReader(f)
            info = pdf.getDocumentInfo()
            created_at = str(info.creation_date)
            created_at_string = created_at[0:10]
            changed_at = str(info.modification_date)
            changed_at_string = changed_at[0:10]
            print(filename + ' Created at: ' + created_at_string +
                  ' Changed at: ' + changed_at_string)
            if info.author == None:
                author = 'Not specified'
            else:
                author = info.author


           
            #get image
            pdf_file = fitz.open(os.path.join(current_path, filename) + '.pdf')
            page = pdf_file[0]
            image_page = page.get_pixmap()
            image_page.shrink(2)
            # image_page.pixel(50,50)
            print(image_page.height, image_page.width)
            # image_page.save(f"page.png")
            image_page.save(f'../image/{filename}.png')
            filePath = os.path.join(sys.path[0], filename+".png")

            # with open(filePath, "rb") as img:

            #     encoded_image = base64.b64encode(img.read())
            #     encoded_image_string = encoded_image.decode('ascii')

            # image_list = page.get_images()
            
            # for image_index, img in enumerate(page.get_images(), start=0):
            #     xref = img[0]
                
            #     base_image = pdf_file.extract_image(xref)
            #     image_bytes = base_image['image']
            #     stream = io.BytesIO(image_bytes)
            #     img2 = Image.open(stream)
            #     # img2.size(50,50)
            #     print(img2.height, img2.width)
            #     img2.show()
            #     image_sequence = img2.getdata()
            #     # image_array = np(image_sequence)
            #     image_array = np.asarray(image_sequence)
            #     image_Ext = base_image['ext']
            #     break
            fileUrl = 'https://github.com/sahinis/hana-enterprise-search-engine/blob/13308eb72f77e91834ac624612eb8408ae66fa21/demo/examples/DocResearch/image/'+ filename +'.png'
            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {   
                        'image': fileUrl,# 'https://png2.cleanpng.com/sh/4a8e8bb378672bb959808405de55e60d/L0KzQYm3U8IxN6F9iZH0aYP2gLBuTfNwdaF6jNd7LXnmf7B6Tflkd59rgdDtZYKweMr3hgJtcZ9wRdR1YXPuPb3wjvsucZR0hp8AYXKzcoG8UvRjO2I3SZC8NEC5QYW9U8E2OmI6SqoEMUS7SYiBTwBvbz==/kisspng-computer-icons-iconfinder-hyperlink-black-link-icon-5ab0b052db3121.3406146315215289148978.png',#fileUrl,#encoded_image_string,
                        'title': filename,
                        'author': author,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)

else:
    print('No folder selected for PDF files')


AUDI_report Created at: 2022-10-27 Changed at: 2022-11-07
68 120
BMW_report Created at: 2022-11-02 Changed at: 2022-11-02
75 106
MERCEDES_report Created at: 2022-03-08 Changed at: 2022-03-10
106 75


Xref table not zero-indexed. ID numbers for objects will be corrected.


PORSCHE_report Created at: 2021-03-21 Changed at: 2021-03-22
106 75
TESLA_report Created at: 2022-10-19 Changed at: 2022-10-19
112 198
TOYOTA_report Created at: 2021-08-23 Changed at: 2021-08-23
106 75
VW_financial_services_report Created at: 2022-03-23 Changed at: 2022-04-06
101 77


## Post data to server

In [17]:
for element in payload_list:
    response = requests.request("POST", f'{base_url}/v1/data/DocResearchDemoExample', headers=headers, data=element)

    print(response.text)

{"Document":[{"id":"cce7a2cf-61c0-11ed-a281-88d82e07e733"}]}
